# LoggerWrapper customization

In this notebook, we will demonstrate how to customize the `LoggerWrapper` provided by *Sinergym*.

In [15]:
import gymnasium as gym
import numpy as np

import sinergym
from sinergym.utils.wrappers import (
    BaseLoggerWrapper,
    CSVLogger,
    LoggerWrapper,
    WandBLogger,
)

## Step 1: Inherit and complete abstract methods from BaseLoggerWrapper

We simply need to inherit from this class and define both the custom metrics to be monitored, and the summary metrics that are calculated from the logger data for each simulated episode. 

Additionally, you can change the back-end where the information is stored by modifying `logger_class`, instead of using the default. 

*Sinergym* use this structure to implement its default [LoggerWrapper](https://ugr-sail.github.io/sinergym/compilation/main/pages/wrappers.html#loggerwrapper).

In [16]:
from typing import Any, Callable, Dict, List, Optional, Union

from sinergym.utils.constants import LOG_WRAPPERS_LEVEL
from sinergym.utils.logger import LoggerStorage, TerminalLogger


class CustomLoggerWrapper(BaseLoggerWrapper):

    logger = TerminalLogger().getLogger(name='WRAPPER CustomLoggerWrapper',
                                        level=LOG_WRAPPERS_LEVEL)

    def __init__(
            self,
            env: gym.Env,
            logger_class: Callable = LoggerStorage):

        super(CustomLoggerWrapper, self).__init__(env, logger_class)
        # Custom variables and summary variables
        self.custom_variables = ['custom_variable1', 'custom_variable2']
        self.summary_variables = ['episode_num',
                                  'double_mean_reward', 'half_power_demand']

    # Define abstract methods for metrics calculation

    def calculate_custom_metrics(self,
                                 obs: np.ndarray,
                                 action: Union[int, np.ndarray],
                                 reward: float,
                                 info: Dict[str, Any],
                                 terminated: bool,
                                 truncated: bool):
        # Variables combining information
        return [obs[0]*2, obs[-1]+reward]

    def get_episode_summary(self) -> Dict[str, float]:
        # Get information from logger
        power_demands = [info['total_power_demand']
                         for info in self.data_logger.infos[1:]]

        # Data summary
        data_summary = {
            'episode_num': self.get_wrapper_attr('episode'),
            'double_mean_reward': np.mean(self.data_logger.rewards)*2,
            'half_power_demand': np.mean(power_demands)/2,
        }
        return data_summary

Low-level changes can be done to the logging system by creating your own ``BaseLoggerWrapper``. This would require a deep understanding of the tool.

## Step 2: Use CustomLoggerWrapper to save information

Now we can combine the new wrapper with any of *Sinergym* 's output wrappers, and the data will be saved properly.

For instance, let's combine it with [CSVLogger](https://ugr-sail.github.io/sinergym/compilation/main/pages/wrappers.html#csvlogger) to save the data in CSV files. However, it can also be used with [WandBLogger](https://ugr-sail.github.io/sinergym/compilation/main/pages/wrappers.html#wandblogger) or any other logger created by the user.

In [17]:
env = gym.make('Eplus-demo-v1')
env = CustomLoggerWrapper(env)
env = CSVLogger(env)

#==============================================================================================#  
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.                                
[ENVIRONMENT] (INFO) : Name: demo-v1                                                  
#==============================================================================================#  
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/demo-v1-res1
[MODEL] (INFO) : Model Config is correct.                                             
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names.  
[MODEL] (INFO) : Updated building model Output:Meter with meter names.                
[MODEL] (INFO) : Building configuration: runperiod updated to {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 1, 'end_month': 3, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 1}
[MODEL] (INFO) : Updated episode length (seconds): 5184000.0            

Now, if we run the environment (with a random agent, for example), we can see how the files are correctly saved in the *Sinergym* output.

`progress.csv` contains the summary variables we have defined, and within the monitor folder of each episode, a new CSV file named `custom_metrics.csv` is created, registering the new metrics tracked.

In [18]:
for i in range(1):
    obs, info = env.reset()
    rewards = []
    truncated = terminated = False
    current_month = 0
    while not (terminated or truncated):
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print('Episode ', i, 'Mean reward: ', np.mean(
        rewards), 'Cumulative reward: ', sum(rewards))
env.close()

#----------------------------------------------------------------------------------------------#  
[ENVIRONMENT] (INFO) : Starting a new episode.                                        
[ENVIRONMENT] (INFO) : Episode 1: demo-v1                                             
#----------------------------------------------------------------------------------------------#  
[MODEL] (INFO) : Episode directory created.                                           
[MODEL] (INFO) : Weather file USA_PA_Pittsburgh-Allegheny.County.AP.725205_TMY3.epw used.
                                                                                                  

[MODEL] (INFO) : Adapting weather to building model.                                  
[ENVIRONMENT] (INFO) : Saving episode output path in /workspaces/sinergym/examples/demo-v1-res1/episode-1/output.
[SIMULATOR] (INFO) : handlers initialized.                                            
[SIMULATOR] (INFO) : handlers are ready.                                              
[SIMULATOR] (INFO) : System is ready.                                                 
[ENVIRONMENT] (INFO) : Episode 1 started.                                             
Simulation Progress [Episode 1]: 100%|██████████| 100/100 [00:00<00:00, 103.65%/s, 100% completed]Episode  0 Mean reward:  -1963.9963889482085 Cumulative reward:  -2828154.8000854207
[WRAPPER CSVLogger] (INFO) : Environment closed, data updated in monitor and progress.csv.
Simulation Progress [Episode 1]: 100%|██████████| 100/100 [00:03<00:00, 32.82%/s, 100% completed] 
[ENVIRONMENT] (INFO) : Environment closed. [demo-v1]                          